In [6]:
import pandas as pd
from sklearn import svm # SVM Package
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix

In [8]:
## Preparing Dataset!

df_main = pd.read_csv("features_032422.csv") # Import features set

# Making dictionary between incident id and category
df_raw = pd.read_csv('raw-data/har_raw.gz')

incident_dict = dict()

for i in range(df_raw.shape[0]):
    incident_dict[df_raw.iloc[i,1]] = df_raw.iloc[i,2]

# Features that are going to be "ignored" based on previous findings

filter_feat = ["mean y","window -7480:-2480 mean y","window -4980:20 mean y","window -2480:2520 mean y",
               "window 20:5020 mean y","window 2520:7520 mean y","peak y"]

df_main.drop(filter_feat, axis=1, inplace=True)
    
# z-score normalizes each feature to make sure the scales are the same (assume normal distribution)
for feature in df_main.columns:
    # Please do not normalize the incident_id values!
    if feature == "incident_id":
        continue
    df_main[feature] = (df_main[feature] - df_main[feature].mean())/df_main[feature].std()

# Separated dataframes for STF and Others
others = [ID for ID, motion in incident_dict.items() if motion == "other"]
stf = list(set(df_main["incident_id"])-set(others)) # Slip, trip, or fall IDs

df_stf = df_main[df_main["incident_id"].isin(stf)].reset_index().drop(labels=["index","incident_id"],axis=1)
df_other =df_main[df_main["incident_id"].isin(others)].reset_index().drop(labels=["index","incident_id"],axis=1)

# Remove the dataframes that will not be used (for RAM purposes)
del others; del stf; del df_main; del df_raw;

In [9]:
def iter_SVM(df_other, df_stf, best_feat, iteration, n_other=df_stf.shape[0], n_stf=df_stf.shape[0]):
    '''
    # This function is meant to iterate through the feature list and apply forward feature selection to build
    # the best model that maximizes the model evaluation metric
    
    df_other (pd.DataFrame): dataframe containing all other-classified events
    df_stf (pd.DataFrame): dataframe containing all slip/trip/fall-classified events
    iteration (int): number of sampling iterations to find the best combination of features
    n_other (int): number of "other" data points to use to train model
    n_stf (int): number of "slip/trip/fall" data points to use to train model
    best_feat (list): labels of previous features that maximized the given training metric
    '''

    def _metric(df_stf,X_valid):
        '''
        # This is function is meant to generate whatever model selection metric that we want to base feature selection
        # off. Right now, it is implementing Positive Predictive Values (PPV; 02 24 2022)

        X_valid (multi-dimension array): contains data from the validation set
        '''
        return(df_stf.shape[0]/np.sum(lin_clf.predict(X_valid)))

    test_feat = list(set(df_stf.columns)-set(best_feat)) # list of features that we are testing
    
    # Dataframe that holds the feature evaluation metric for each iteration
    metric = pd.DataFrame(zip(test_feat,np.zeros(len(test_feat))),
                          columns=["feature","score1"]).set_index("feature")
        
    for reps in range(iteration):
        ### Want to use the same testing and validation set between evaluating features
       
        # Gets the row indices for the training data sets
        train_other = resample([i for i in range(df_other.shape[0])],n_samples=n_others) 
        train_stf = resample([i for i in range(df_stf.shape[0])],n_samples=n_stf)

        # Gets the row indices for the validation training set
        valid_other = [i for i in range(df_other.shape[0]) if i not in train_other]
        valid_stf = [i for i in range(df_stf.shape[0]) if i not in train_stf]
        
        _rep = "score" + str(reps+1)
        metric = pd.concat([metric,pd.DataFrame(np.zeros(metric.shape[0]),columns=[_rep])],axis=1)
        
        for trial in test_feat:
            feature = best_feat + [trial] # feature list to train SVM with
            
            # Builds an feature list (X) and a list with their respective labels (Y).
            # For Y "values, 0 = others and 1 = Slip/Trip/Fall
            train_X = np.append(df_other.iloc[train_other,:][feature].to_numpy(), df_stf.iloc[train_stf,:][feature].to_numpy())
            train_Y = np.append(np.zeros(n_other),np.full((1,n_stf),1))
            valid_X = np.append(df_other.iloc[valid_other,:][feature].to_numpy(), df_stf.iloc[valid_stf,:][feature].to_numpy())
            
            train_X = train_X.reshape(-1,len(feature))
            valid_X = valid_X.reshape(-1,len(feature))
            
            # Training the Linear SVM
            lin_clf = svm.LinearSVC(dual=False,C=1,max_iter=10**5)
            lin_clf.fit(train_X,train_Y)
            
            # Adds performance metrix to the DataFrame
            metric.loc[trial,_rep] = _metric(df_stf,valid_X)
    
    avg_metric = metric.mean(axis=1) # averages metrics for the same feature
    maxed_feat = avg_metric.idxmax() # finds the best feature with the greatest averaged metric
    
    return (maxed_feat,metric)

def confusion_matrix():
    '''
    Creates a confusion matrix from the true_y and predicted_y values. Still working on it...
    '''
    return

The block of code below just finds the first "couple" top features that would be selected sequentially based on what maximized the performance metric PPV.

In [5]:
# Tunable Parameters
iterations = 100
n_others = 200 # number of other data points to train model with (default: df_stf.shape[0], matches maximum positive cases)
n_stf = 200 # number of slip, trip, fall data points to train model with (default: df_stf.shape[0], matches maximum positive cases)

PPV = [0] # Holds the PPV values as we successively add features using forward feature selection; needs to initialize with 0 to begin comparisons
best_feat = [] # Hold the features we add in chronological order

count = 0

# Loop iterates through all the features, and breaks when the PPV no longer increases (onset of overfitting)
for i in range(df_stf.shape[1]):
    top_feat, df_PPV = iter_SVM(df_other,df_stf,best_feat,iterations,n_others, n_stf)
    
    best_feat.append(top_feat)
    PPV.append(df_PPV.loc[top_feat].mean(axis=0))
    
    print("Best Feature: %s, PPV: %.4f" %(top_feat,PPV[i+1]))
    
    count += 1
    if count == 25:
        break

Best Feature: window -2480:2520 low frequency power, PPV: 0.4283
Best Feature: window 2520:7520 low frequency power, PPV: 0.3770
Best Feature: window 20:5020 low frequency power, PPV: 0.3564
Best Feature: window -2480:2520 medium frequency power, PPV: 0.3209
Best Feature: medium frequency power, PPV: 0.3066
Best Feature: window -4980:20 medium frequency power, PPV: 0.3055
Best Feature: window 20:5020 kurtosis, PPV: 0.2833
Best Feature: window -2480:2520 high frequency power, PPV: 0.2824
Best Feature: mean x, PPV: 0.2731
Best Feature: window -4980:20 kurtosis, PPV: 0.2786
Best Feature: window 20:5020 minimum, PPV: 0.2781
Best Feature: window 20:5020 range, PPV: 0.2792
Best Feature: window -4980:20 mean z, PPV: 0.2756
Best Feature: window 20:5020 mean x, PPV: 0.2761
Best Feature: kurtosis, PPV: 0.2793
Best Feature: peak x, PPV: 0.2822
Best Feature: peak z, PPV: 0.2753
Best Feature: window 20:5020 maximum, PPV: 0.2789
Best Feature: window -7480:-2480 medium frequency power, PPV: 0.2787
Be

AttributeError: 'list' object has no attribute 'figsize'

In [32]:
#PPV.pop(0) # Uncomment this if you are running it for the first time
plt.rcParams.update({'font.size': 15})
plt.plot([i+1 for i in range(25)],PPV)
plt.xlabel("Number of Features used in the Model")
plt.ylabel("Positive Predictive Value")
plt.title("Forward Feature Selection using a Linear Kernel SVM")
fig.figsize([9,6])

plt.savefig("Sample_Plot.png",dpi=600)

This block of code below just does pure forward feature selection and stops when the performance metric (PPV) does not increase.

In [20]:
# Tunable Parameters
iterations = 300
n_others = 200 # number of other data points to train model with (default: df_stf.shape[0], matches maximum positive cases)
n_stf = 200 # number of slip, trip, fall data points to train model with (default: df_stf.shape[0], matches maximum positive cases)

PPV = [0] # Holds the PPV values as we successively add features using forward feature selection; needs to initialize with 0 to begin comparisons
best_feat = [] # Hold the features we add in chronological order

# Loop iterates through all the features, and breaks when the PPV no longer increases (onset of overfitting)
for i in range(df_stf.shape[1]):
    top_feat, df_PPV = iter_SVM(df_other,df_stf,best_feat,iterations,n_others, n_stf)
    
    best_feat.append(top_feat)
    PPV.append(df_PPV.loc[top_feat].mean(axis=0))
    
    print("Best Feature: %s, PPV: %.4f" %(top_feat,PPV[i+1]))

    # Checks if the new added feature increases PPV; else, break from loop
    if PPV[i+1] < PPV[i]:
        print("\nDone!")
        break

Best Feature: window 2520:7520 low frequency power, PPV: 0.4339
Best Feature: window -2480:2520 low frequency power, PPV: 0.3844

Done!
